Uber Supply and Demand Case Study 
What are the insights we find?

1.Which date had the most completed trips during the two weeks?

2.What was the highest number of completed trips within a 24-hour period?

3.Which hour of the day had the most requests during the two-week period?

4.What percentages of all zeroes during the two week period occurred on weekends (Friday at 5 pm to Sunday at 3 am)?

5.What is the weighted average ratio of completed trips per driver during the two week period?

6.In drafting a driver schedule in terms of 8 hour shifts, when are the busiest 8 consecutive hours over the two week period in terms of unique requests? A new shift starts every 8 hours. Assume that a driver will work the same shift each day.

7.True or False: Driver supply always increases when demand increases during the two week period.

8.In which 72 hour period is the ratio of Zeroes to Eyeballs the highest?

9.If you could add 5 drivers to any single hour of every day during the two week period, which hour should you add them to?

10.True or False: There is exactly two weeks of data in this analysis

11.Looking at the data from all two weeks, which time might make the most sense to consider a true "end day" instead of midnight? (i.e when are supply and demand at both their natural minimums)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df =pd.read_csv("C:/Stevens/Insights from City Supply and Demand Data/dataset.csv")

In [4]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,10-Sep-12,8,6,0,2,2,14
2,10-Sep-12,9,8,3,0,0,14
3,10-Sep-12,10,9,2,0,1,14
4,10-Sep-12,11,11,1,4,4,11


Checking the Data


In [5]:
df.isnull().sum()

Date                0
Time (Local)        0
Eyeballs            0
Zeroes              0
Completed Trips     0
Requests            0
Unique Drivers      0
dtype: int64

## 1) Which date had the most completed trips during the two week period?

In [6]:
print(df.columns)

Index(['Date', 'Time (Local)', 'Eyeballs ', 'Zeroes ', 'Completed Trips ',
       'Requests ', 'Unique Drivers'],
      dtype='object')


In [7]:
trips_made = df.groupby('Date')['Completed Trips '].sum()

date_most_trips = trips_made.idxmax()
total_trips = trips_made.max()
print(f"Date with the most completed trips: {date_most_trips} with total trips as {total_trips}")


Date with the most completed trips: 22-Sep-12 with total trips as 248


## 2) What was the highest number of completed trips within a 24 hour period?

In [8]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,10-Sep-12,8,6,0,2,2,14
2,10-Sep-12,9,8,3,0,0,14
3,10-Sep-12,10,9,2,0,1,14
4,10-Sep-12,11,11,1,4,4,11


In [9]:
max_trips = trips_made.max()
print(f"Highest number of completes trips within a 24 hours: {max_trips}")

Highest number of completes trips within a 24 hours: 248


## 3) Which hour of the day had the most requests during the two week period?

In [10]:
request_by_hr = df.groupby('Time (Local)')['Requests '].sum()

most_request = request_by_hr.idxmax()
print(f"Hour with the most request: {most_request}:00 ")

Hour with the most request: 23:00 


## 4) What percentages of all zeroes during the two week period occurred on weekend (Friday at 5 pm to Sunday at 3 am)? Tip: The local time value is the start of the hour (e.g. '15' is the hour from 3:00pm - 4:00pm)

In [11]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,10-Sep-12,8,6,0,2,2,14
2,10-Sep-12,9,8,3,0,0,14
3,10-Sep-12,10,9,2,0,1,14
4,10-Sep-12,11,11,1,4,4,11


In [12]:
df['datetime'] = pd.to_datetime( df['Date'] + ' ' + df['Time (Local)'].astype(str) + ':00')

weekend = (
    (df['datetime'].dt.dayofweek == 4) & (df['datetime'].dt.hour >=17) |
    (df['datetime'].dt.dayofweek == 5) |
    (df['datetime'].dt.dayofweek == 6) & (df['datetime'].dt.hour < 3) 
)

total_zeroes = df['Zeroes '].sum()
weekend_zeroes= df.loc[weekend, 'Zeroes '].sum()

percentage_of_zeroes_weekend = (weekend_zeroes/total_zeroes)*100
print(f"Percentage of zeroes on weekend: {percentage_of_zeroes_weekend:.2f}%")
                                

Percentage of zeroes on weekend: 44.86%


C:\Users\navne\AppData\Local\Temp\ipykernel_21484\258819708.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime( df['Date'] + ' ' + df['Time (Local)'].astype(str) + ':00')


## 5) What is the weighted average ratio of completed trips per driver during the two week period? Tip: "Weighted average" means your answer should account for the total trip volume in each hour to determine the most accurate number in whole period.

In [13]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,datetime
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00


In [14]:
df['trips_per_driver'] = df['Completed Trips '] / df['Unique Drivers']

weighted_avg = (df['trips_per_driver'] * df['Unique Drivers']).sum()/df['Unique Drivers'].sum()

print(f"Weighted average ratio of completed trips per driver: {weighted_avg:.2f}")

Weighted average ratio of completed trips per driver: 0.51


## 6) In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two week period in terms of unique requests? A new shift starts in every 8 hours. Assume that a driver will work same shift each day.

In [15]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,datetime,trips_per_driver
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0.222222
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0.142857
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0.000000
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0.000000
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0.363636


In [16]:
hourly_unique_requests = df.groupby(df['datetime'].dt.hour)['Requests '].nunique()

busiest_8hours_start = hourly_unique_requests.rolling(window=8).sum().idxmax()

busiest_8hours_start = pd.to_datetime(busiest_8hours_start, format='%H')
busiest_8hours_end = busiest_8hours_start + pd.Timedelta(hours=8)

print(f"The busiest 8-hour period in the two-week period is {busiest_8hours_start.time()} to {busiest_8hours_end.time()} hours")


The busiest 8-hour period in the two-week period is 22:00:00 to 06:00:00 hours


## 7) True or False: Driver supply always increases when demand increases during the two week period. Tip: Visualize the data to confirm your answer if needed.

In [18]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,datetime,trips_per_driver
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0.222222
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0.142857
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0.000000
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0.000000
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0.363636


In [20]:
correlation = df['Unique Drivers'].corr(df['Requests '])
print(f"Driver supply always increases with demand: {correlation>0}")

Driver supply always increases with demand: True


In [24]:
demand_supply = df.groupby('Date').agg({'Requests ': 'sum', 'Unique Drivers': 'sum'}).reset_index()

inc_supply = (demand_supply['Unique Drivers'].diff() > 0 ) & (demand_supply['Requests '].diff() > 0)

supply_increase_with_demand = inc_supply.all()

print(f"Driver supply always increase when demand increases : {supply_increase_with_demand}")
if not supply_increase_with_demand:
    print("The demand for Uber request does not guarantee an increase in the supply for drivers")

Driver supply always increase when demand increases : False
The demand for Uber request does not guarantee an increase in the supply for drivers


## 8) In which 72 hour period is the ratio of Zeroes to Eyeballs the highest?

In [27]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time (Local)'].astype(str) + ':00')

df['zeroes_eyeball'] = df['Zeroes '] / df['Eyeballs ']

df['roll_ratio'] = df['zeroes_eyeball'].rolling(window=72).mean()

highest_ratio_start = df.loc[df['roll_ratio'].idxmax(), 'datetime']
highest_ratio_value = df['roll_ratio'].max()

print(f"The 72-hour period with highest zeroes to eyeball ratio starts at {highest_ratio_start} with ratio of {highest_ratio_value:.2f}")

The 72-hour period with highest zeroes to eyeball ratio starts at 2012-09-18 04:00:00 with ratio of 0.37


C:\Users\navne\AppData\Local\Temp\ipykernel_21484\2702372541.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time (Local)'].astype(str) + ':00')


## 9) If you could add 5 drivers to any single hour of every day during the two week period, which hour should you add them to? Hint: Consider both rider eyeballs and driver supply when choosing

In [28]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,datetime,trips_per_driver,zeroes_eyeball,roll_ratio
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0.222222,0.000000,NaN
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0.142857,0.000000,NaN
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0.000000,0.375000,NaN
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0.000000,0.222222,NaN
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0.363636,0.090909,NaN


In [29]:
df['D_S_gap'] = df['Requests '] - df['Unique Drivers']
hr_add_drivers= df.groupby('Time (Local)')['D_S_gap'].mean().idxmax()
print(f"Add 5 drivers to hour : {hr_add_drivers}:00")

Add 5 drivers to hour : 23:00


Here we are addressing the largest absolute shortage.

## 10) True or False: There is exactly two weeks of data in this analysis

In [30]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,datetime,trips_per_driver,zeroes_eyeball,roll_ratio,D_S_gap
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0.222222,0.000000,NaN,-7
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0.142857,0.000000,NaN,-12
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0.000000,0.375000,NaN,-14
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0.000000,0.222222,NaN,-13
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0.363636,0.090909,NaN,-7


In [33]:
date_range = df['datetime'].max() - df['datetime'].min()
print(f" There is exactly two weeks of data : {date_range == 14}" )

# here we are also considering time. For example 14 days, 23 hours, 59 minutes 

 There is exactly two weeks of data : False


## 11) Looking at the data from all two weeks, which time might make the most sense to consider a true "end day" instead of midnight? (i.e when are supply and demand at both their natural minimums) Tip: Visualize the data to confirm your answer if needed.

In [35]:
min_req_hr = df.groupby('Time (Local)')['Requests '].mean().idxmax()

min_drivers_hr = df.groupby('Time (Local)')['Unique Drivers'].mean().idxmax()

print(f"Hour with the lowest demand (requests): {min_req_hr}:00")
print(f"Hour with the lowest supply (drivers): {min_drivers_hr}:00")

Hour with the lowest demant (requests): 23:00
Hour with the lowest supply (drivers): 19:00


In [37]:
avg_hr_completed_trips = df.groupby('Time (Local)')['Completed Trips '].mean()
avg_hr_unique_drivers = df.groupby('Time (Local)')['Unique Drivers'].mean()

hr_lowest_D_S = (avg_hr_completed_trips + avg_hr_unique_drivers).idxmin()
lowest_D_S_value = (avg_hr_completed_trips + avg_hr_unique_drivers).min()

print(f'We consider {hr_lowest_D_S}:00 as the true "end day" as the supply of drivers and demand sre at natural minimum with a value of {lowest_D_S_value:.2f}')

We consider 4:00 as the true "end day" as the supply of drivers and demand sre at natural minimum with a value of 0.79
